# Setting up ADK 

In [1]:
pip install google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error:{e}"
    )

✅ Gemini API key setup complete.


In [3]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
retry_config=types.HttpRetryOptions(
    attempts=5, 
    exp_base=7,  
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], 
)

# Multi agents

Acts a team of simple, specialized agents that collaborate, just like a real-world team. Each agent has one clear job (e.g., one agent only does research, another only writes). This makes them easier to build, easier to test, and much more powerful and reliable when working together.

Example: Research & Summarization System


Research Agent - Searches for information using Google Search
Summarizer Agent - Creates concise summaries from research findings


In [5]:
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""specialized research agent.job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

✅ research_agent created.


In [6]:
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
   
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


## we bring the agents together under a root agent, or coordinator
Orchestrates the workflow by calling the sub-agents as tools.

In [7]:
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ root_agent created.")


✅ root_agent created.


## Run

In [8]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Can you explain me why it is important to go to college?"
)



 ### Created new session: debug_session_id

User > Can you explain me why it is important to go to college?


ResearchCoordinator > Going to college is important for several key reasons:

*   **Expanded Job Opportunities:** A college degree opens more career doors and better equips graduates to compete in the job market, with many professions requiring a bachelor's for entry.
*   **Higher Earning Potential:** College graduates typically earn significantly more over their lifetime, leading to greater financial stability and improved quality of life.
*   **Personal Development:** College fosters critical thinking, problem-solving skills, adaptability, independence, and self-discovery, while also building supportive relationships.
*   **Increased Community Engagement:** College graduates often show higher rates of civic participation, including voting, volunteering, and community involvement.


# "assembly line" using a sequential agent,
The previous multi-agent system worked, but it relied on a detailed instruction prompt to force the LLM to run steps in order. This can be unreliable. A complex LLM might decide to skip a step, run them in the wrong order, or get "stuck," making the process unpredictable.

When you need tasks to happen in a guaranteed, specific order, you can use a SequentialAgent. This agent acts like an assembly line, running each sub-agent in the exact order you list them. The output of one agent automatically becomes the input for the next, creating a predictable and reliable workflow.

# Blog post creation

In [9]:
outline_agent = Agent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline",  
)

print("✅ outline_agent created.")

✅ outline_agent created.


In [10]:
writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft",  
)

print("✅ writer_agent created.")


✅ writer_agent created.


In [11]:
editor_agent = Agent(
    name="EditorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),

    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog",  
)

print("✅ editor_agent created.")

✅ editor_agent created.


## Sequential agent:
runs it in order

In [12]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


In [13]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a blog post about why college isnt really that beneficial"
)


 ### Created new session: debug_session_id

User > Write a blog post about why college isnt really that beneficial
OutlineAgent > Okay, I am OutlineAgent. Here is a blog post outline on the topic "Why College Isn't Really That Beneficial":

## Blog Post Outline:

**Catchy Headline:** The Myth of the Golden Ticket: Why Your College Degree Might Be Worth Less Than You Think

**Introduction Hook:** Remember the endless pressure to get good grades, ace the SATs, and get into a "good" college? We're told it's the guaranteed path to success, a golden ticket to a fulfilling career and financial security. But what if that narrative is outdated? What if, for many, the supposed benefits of a four-year degree are increasingly overshadowed by mounting costs and questionable returns?

**Main Section 1: The Crushing Weight of Debt**

*   **Bullet Point 1:** Student loan debt has reached astronomical levels, burdening graduates for decades and impacting major life decisions like buying a home, start

# Parallel Workflows - Independent Researchers
several tasks that are not dependent on each other
When you have independent tasks, you can run them all at the same time using a ParallelAgent. This agent executes all of its sub-agents concurrently, dramatically speeding up the workflow. Once all parallel tasks are complete, you can then pass their combined results to a final 'aggregator' step.



In [14]:
tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",  
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [15]:
health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research",  # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [16]:
finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",  
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


In [17]:
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
   
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary",  
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


In [18]:
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


In [19]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "explain why college isnt really that beneficial"
)


 ### Created new session: debug_session_id

User > explain why college isnt really that beneficial
FinanceResearcher > While college education offers numerous benefits, including increased job opportunities and higher earning potential, it's not universally beneficial and presents significant drawbacks.

One major downside is the **financial burden**. College costs, including tuition, fees, and living expenses, can lead to substantial student debt that impacts graduates for years. The return on investment (ROI) for degrees varies greatly by field, with some humanities or liberal arts degrees offering little to no financial payoff.

Another concern is the **opportunity cost**, as the time spent in college could be used to gain practical, real-world experience or pursue alternative career paths that might be faster or more affordable. Furthermore, a degree doesn't guarantee employment; many graduates end up underemployed in jobs that don't require their qualifications. The value employe

# Loop Workflows 
A LoopAgent runs a set of sub-agents repeatedly until a specific condition is met or a maximum number of iterations is reached. This creates a refinement cycle, allowing the agent system to improve its own work over and over.

In [20]:
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story",  # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [21]:
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique", 
)

print("✅ critic_agent created.")

✅ critic_agent created.


A simple Python function that the LoopAgent understands as an "exit" signal.
An agent that can call that function when the right condition is met.

In [22]:
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}


print("✅ exit_loop function created.")


✅ exit_loop function created.


In [23]:
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    output_key="current_story",  
    tools=[
        FunctionTool(exit_loop)
    ],  
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


In [24]:
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # Prevents infinite loops
)


root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [25]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a short story about a dog"
)


 ### Created new session: debug_session_id

User > Write a short story about a dog
InitialWriterAgent > Buster loved Tuesdays. Tuesdays meant the park, and the park meant the big, slobbery Labrador named Gus. Today, however, Gus wasn't there. Buster whined, his tail giving a pathetic little thump. He sniffed the familiar patch of grass where Gus usually greeted him with a happy bark. Nothing. A lone squirrel darted up an oak, taunting him. Buster, usually more interested in chasing squirrels than existential dread, felt a pang of genuine sadness. He trotted towards the park bench where his human always sat, nudging her hand. Maybe, just maybe, she knew where Gus was.
CriticAgent > 1. **Show, Don't Tell Buster's Sadness:** Instead of stating Buster felt "genuine sadness," describe his physical reactions more vividly. For example, "His ears drooped, and a low rumble vibrated in his chest, a sound he usually reserved for thunderstorms."

2. **Develop the Human's Role:** Buster's human is